In [5]:
import numpy as np
import classy

import fastmapsvm as fm
import h5py
import sklearn.metrics

# Setup Training Data

In [6]:
def random_sample(n_samples, param_range):
    pmin = param_range[0]
    pmax = param_range[1]
    return np.random.uniform(pmin, pmax, n_samples) 

In [7]:
n_params = 5
n_samples = 1000 

omega_b_sample = random_sample(n_samples, [0.01875, 0.02625])
omega_cdm_sample = random_sample(n_samples, [0.05, 0.255])
h_sample = random_sample(n_samples, [0.64, 0.82])
ns_sample = random_sample(n_samples, [0.84, 1.1])
ln_As_sample = random_sample(n_samples, [1.61, 3.91])

input_parameters = np.stack((omega_b_sample, omega_cdm_sample, h_sample, ns_sample, ln_As_sample), axis=-1)

In [8]:
import time
# output_spectra
kmin = 1e-5
kmax = 1e1
n_k = 420
k_arr = np.geomspace(kmin, kmax, n_k)
output_spectra = np.zeros((n_samples, n_k))

# t = time.time()
for i in range(n_samples):
    param = input_parameters[i]
    param_dict = {
        'omega_b': param[0], 
        'omega_cdm': param[1], 
        'h': param[2], 
        'n_s': param[3], 
        'A_s': 10**(param[4])/1.0e10,
        'output': 'mPk',
        'P_k_max_1/Mpc': kmax,
    
    }
    cosmo = classy.Class()
    cosmo.set(param_dict)
    cosmo.compute()
    output_spectra[i,:] = [cosmo.pk_lin(k, 0) for k in k_arr] # just doing z=0 for now
# print(f'took {time.time()-t} s')

In [9]:
np.savez_compressed(f'test_nsamples_{n_samples}', parameters=input_parameters, spectra=output_spectra)

# Create P(k) Model 

In [ ]:
model_path = "CLASS_model.hdf5" # Model output path.
data_path  = "test_nsamples_{n_samples}"       # Test/training data path.

# Define a distance function on pairs of objects.
def correlation_distance(a, b):
    """
    Returns the correlation distance between objects a and b.
    """
    return (
        1 - np.max(np.abs(np.mean(fm.distance.ndcorrelate(a, b, mode="same"), axis=0)))
    )